In [1]:
# --- 1. Импорты и общие настройки ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet  # pip install prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings("ignore")
# plt.style.use("seaborn-whitegrid")



In [3]:
# --- 2. Загрузка данных ---
# df должен содержать столбцы: "Регион", "Период" (YYYY-MM), и целевой столбец с показателем.
df = pd.read_excel("Датасет по птице.xlsx")
df["Период"] = pd.to_datetime(df["Период"], format="%Y-%m")
df.sample(10)



,Регион,Период,Птица
1365,КОСТАНАЙСКАЯ ОБЛАСТЬ,2022-03-01,1117.97
495,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,2016-04-01,4125.18
1421,КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ,2016-11-01,31.90
1301,КОСТАНАЙСКАЯ ОБЛАСТЬ,2016-11-01,455.00
1726,ОБЛАСТЬ ҰЛЫТАУ,2024-07-01,9.60
129,АКТЮБИНСКАЯ ОБЛАСТЬ,2015-10-01,320.37
609,ГАЛМАТЫ,2015-10-01,0.49
1129,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,2022-07-01,1020.36
1791,ПАВЛОДАРСКАЯ ОБЛАСТЬ,2019-12-01,825.50
1407,КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ,2015-09-01,0.66


In [4]:
regions = df['Регион'].unique()
target   = "Птица"
horizon  = 3
epsilon = 1e-6

In [5]:
first_test = pd.to_datetime("2024-01-01")
last_possible = df["Период"].max() - pd.DateOffset(months=horizon-1)
test_starts = pd.date_range(first_test, last_possible, freq="MS")

## Holw-Winter's (log)

In [6]:
results_hw = []
for region in regions:
    ts = (df[df["Регион"] == region]
          .set_index("Период")[target]
          .dropna()
          .sort_index())
    if len(ts) < 24:
        print(f"{region}: всего {len(ts)} мес. — сезонный Holt-Winter's невозможен.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        # формируем train / test
        train = ts[ts.index < test_start]
        test  = ts[(ts.index >= test_start) & (ts.index <= test_end)]

        # пропускаем, если недостаточно данных или неполный test
        if len(train) < 24 or len(test) < horizon:
            continue

        # обучаем модель
        train_log = np.log1p(train)

        hw_log = ExponentialSmoothing(
            train_log,
            seasonal="add",
            seasonal_periods=12
        ).fit(optimized=True)

        # прогноз и метрики
        fc_log = hw_log.forecast(horizon)
        fc = np.expm1(fc_log) 
        # fc   = model.forecast(horizon)
        rmse = np.sqrt(mean_squared_error(test, fc))
        mae  = mean_absolute_error(test, fc)
        mape = (np.abs((test - fc) / test).mean()) * 100

        results_hw.append({
            "Регион":      region,
            "Test start":  test_start.strftime("%Y-%m"),
            "Test end":    test_end.strftime("%Y-%m"),
            "Forecast":    [x.round(2) for x in list(fc.values)],
            "Actual":      [y.round(2) for y in list(test.values)],
            "RMSE":        rmse,
            "MAE":         mae,
            "MAPE_%":      mape
        })

# 4) Усреднение по всем скользящим окнам для каждого региона
res_hw = pd.DataFrame(results_hw)
print("Результаты прогнозов HW на 3 месяца:")

display(res_hw)

final_hw = (
    res_hw
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики Holt–Winter's по регионам (rolling-3):")
display(final_hw)

Результаты прогнозов HW на 3 месяца:


,Регион,Test start,Test end,Forecast,Actual,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,"[8370.33, 8231.05, 8993.66]","[8963.23, 9586.12, 10969.85]",1425.14,1308.05,12.92
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,"[8580.64, 9371.93, 9503.97]","[9586.12, 10969.85, 10894.97]",1353.91,1331.47,12.61
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,"[10018.67, 10153.83, 9794.54]","[10969.85, 10894.97, 10238.97]",741.97,712.25,6.60
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,"[10726.45, 10344.52, 10434.36]","[10894.97, 10238.97, 12692.46]",1308.76,844.06,6.79
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,"[10440.63, 10530.26, 10506.03]","[10238.97, 12692.46, 10379.17]",1255.90,830.24,6.74
...,...,...,...,...,...,...,...,...
180,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,"[576.51, 246.32, 488.39]","[305.92, 742.4, 700.6]",348.49,326.29,61.85
181,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,"[174.65, 348.34, 252.94]","[742.4, 700.6, 711.12]",467.74,459.40,63.73
182,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,"[688.79, 495.89, 635.55]","[700.6, 711.12, 691.74]",128.61,94.41,13.36
183,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,"[499.73, 640.42, 517.96]","[711.12, 691.74, 667.3]",152.34,137.35,19.84


Средние метрики Holt–Winter's по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,1217.82,986.42,9.37
1,АКТЮБИНСКАЯ ОБЛАСТЬ,36.64,30.45,384.26
2,АЛМАТИНСКАЯ ОБЛАСТЬ,885.29,773.67,6.56
3,АТЫРАУСКАЯ ОБЛАСТЬ,0.49,0.41,NaN
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,571.62,515.90,8.36
5,ГАЛМАТЫ,0.06,0.06,54.28
6,ГАСТАНА,0.06,0.05,NaN
7,ГШЫМКЕНТ,40.75,34.22,423.80
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,338.56,294.65,17.50
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,204.40,166.59,16.21


In [ ]:
# res_hw[res_hw['Регион'] == 'АКТЮБИНСКАЯ ОБЛАСТЬ']

,Регион,Test start,Test end,Forecast,Actual,RMSE,MAE,MAPE_%
10,АКТЮБИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,"[61.67, 45.75, 19.9]","[64.3, 56.5, 82.4]",36.64,25.29,32.98
11,АКТЮБИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,"[46.13, 20.07, 45.75]","[56.5, 82.4, 53.6]",36.76,26.85,36.21
12,АКТЮБИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,"[20.91, 47.59, 53.5]","[82.4, 53.6, 37.0]",36.92,28.00,43.47
13,АКТЮБИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,"[62.45, 69.99, 28.91]","[53.6, 37.0, 46.4]",22.15,19.78,47.79
14,АКТЮБИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,"[67.77, 27.98, 24.21]","[37.0, 46.4, 52.6]",26.41,25.86,58.95
15,АКТЮБИНСКАЯ ОБЛАСТЬ,2024-06,2024-08,"[24.75, 21.43, 47.99]","[46.4, 52.6, 68.4]",24.88,24.41,45.25
16,АКТЮБИНСКАЯ ОБЛАСТЬ,2024-07,2024-09,"[24.25, 54.09, 50.95]","[52.6, 68.4, 1.6]",33.88,30.67,1053.01
17,АКТЮБИНСКАЯ ОБЛАСТЬ,2024-08,2024-10,"[63.12, 59.37, 95.38]","[68.4, 1.6, 65.0]",37.81,31.14,1221.76
18,АКТЮБИНСКАЯ ОБЛАСТЬ,2024-09,2024-11,"[60.38, 96.96, 130.58]","[1.6, 65.0, 61.3]",55.61,53.34,1278.61
19,АКТЮБИНСКАЯ ОБЛАСТЬ,2024-10,2024-12,"[53.99, 73.25, 159.68]","[65.0, 61.3, 254.2]",55.37,39.16,24.54


## SARIMA

In [8]:
results_sarima = []

for region in regions:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    ts = ts + epsilon
    ts_log = np.log(ts)

    if len(ts_log) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. для авто-ARIMA, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train_log = ts_log[ts_log.index < test_start]
        test_log  = ts_log[(ts_log.index >= test_start) & (ts_log.index <= test_end)]
        if len(train_log) < 12 + horizon or len(test_log) < horizon:
            continue

        # автоподбор на лог-данных
        use_seasonal = len(train_log) >= 2 * 12

        sarima_log = auto_arima(
            train_log,
            seasonal=use_seasonal,
            m=12 if use_seasonal else 1,
            D=1 if use_seasonal else 0,      # фиксируем порядок сезонной разности
            seasonal_test=None,               # пропустить nsdiffs
            boxcox=True,
            stepwise=True,
            suppress_warnings=True,
            error_action="ignore"
        )
      
        # прогноз в лог-шкале
        fc_log = sarima_log.predict(n_periods=horizon, return_conf_int=False)

        # возвращаем прогноз в исходные единицы
        fc = np.exp(fc_log) - epsilon
        actual = np.exp(test_log.values) - epsilon  # но exp(log(x)) == x

        # метрики на исходном уровне
        rmse = np.sqrt(mean_squared_error(actual, fc))
        mae  = mean_absolute_error(actual, fc)
        mape = (np.abs((actual - fc) / actual).mean()) * 100

        results_sarima.append({
            "Регион":         region,
            "Test start":     test_start.strftime("%Y-%m"),
            "Test end":       test_end.strftime("%Y-%m"),
            "order":          sarima_log.order,
            "seasonal_order": sarima_log.seasonal_order,
            "RMSE":           round(rmse,2),
            "MAE":            round(mae,2),
            "MAPE_%":         round(mape,2),
            "Forecast":       [round(x,2) for x in fc],
            "Actual":         [round(y,2) for y in actual]
        })
# формируем DataFrame с результатами
res_sarima = pd.DataFrame(results_sarima)
print("Результаты прогнозов SARIMA на 3 месяца:")

display(res_sarima)

final_sarima = (
    res_sarima
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики SARIMA по регионам (rolling-3):")
display(final_sarima)

Результаты прогнозов SARIMA на 3 месяца:


,Регион,Test start,Test end,order,seasonal_order,RMSE,MAE,MAPE_%,Forecast,Actual
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,"(3, 1, 0)","(0, 1, 1, 12)",751.48,593.33,5.69,"[8994.06, 8997.39, 9809.41]","[8963.23, 9586.12, 10969.85]"
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,"(3, 1, 0)","(0, 1, 1, 12)",828.43,772.80,7.33,"[8969.3, 9779.4, 10383.83]","[9586.12, 10969.85, 10894.97]"
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,"(3, 1, 0)","(0, 1, 1, 12)",569.84,432.16,4.01,"[10035.51, 10847.31, 10553.45]","[10969.85, 10894.97, 10238.97]"
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,"(3, 1, 2)","(1, 1, 1, 12)",1161.00,902.61,7.61,"[11015.8, 10948.1, 10814.61]","[10894.97, 10238.97, 12692.46]"
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,"(2, 1, 2)","(0, 1, 1, 12)",1054.11,848.43,7.21,"[10377.68, 11031.78, 11125.06]","[10238.97, 12692.46, 10379.17]"
...,...,...,...,...,...,...,...,...,...,...
195,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,"(0, 0, 3)","(0, 1, 1, 12)",414.15,395.35,72.75,"[596.6, 173.85, 373.78]","[305.92, 742.4, 700.6]"
196,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,"(2, 0, 0)","(2, 1, 0, 12)",569.46,565.71,78.61,"[85.29, 191.63, 180.08]","[742.4, 700.6, 711.12]"
197,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,"(2, 0, 0)","(2, 1, 0, 12)",184.52,182.62,26.06,"[554.65, 516.08, 484.87]","[700.6, 711.12, 691.74]"
198,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,"(2, 0, 0)","(2, 1, 0, 12)",186.92,181.51,26.48,"[564.76, 538.1, 422.78]","[711.12, 691.74, 667.3]"


Средние метрики SARIMA по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,1116.18,968.54,9.27
1,АКТЮБИНСКАЯ ОБЛАСТЬ,44.09,36.86,356.69
2,АЛМАТИНСКАЯ ОБЛАСТЬ,1318.64,1170.44,9.93
3,АТЫРАУСКАЯ ОБЛАСТЬ,0.06,0.03,3505499728190416879616.00
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,580.97,549.82,8.74
5,ГАЛМАТЫ,0.22,0.17,168.59
6,ГАСТАНА,0.02,0.01,32418544302525849600.00
7,ГШЫМКЕНТ,83.27,63.01,780.23
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,373.67,325.07,19.32
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,164.23,139.76,13.88


## Facebook Prophet

In [9]:
results_prophet = []

for region in regions:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    if len(ts) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. данных, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train = ts[ts.index < test_start]
        test  = ts[(ts.index >= test_start) & (ts.index <= test_end)]
        if len(train) < 12 + horizon or len(test) < horizon:
            continue

        # Подготовка данных для Prophet
#         df_prophet = train.reset_index().rename(columns={"Период":"ds", target:"y"})
# # подготовка для одного региона
        df_prophet = train.reset_index().rename(columns={"Период":"ds", target:"y"})
        df_prophet["y"] = np.log(df_prophet["y"] + epsilon)

        m = Prophet()
        m.fit(df_prophet)

        future = m.make_future_dataframe(periods=horizon, freq="MS")
        forecast = m.predict(future)

        # берем только прогнозные точки
        yhat_log = forecast["yhat"].values[-horizon:]
        fc = np.exp(yhat_log) - epsilon

        # m = Prophet()
        # m.fit(df_prophet)

        # # Создаем DataFrame будущих дат и делаем прогноз
        # # future = m.make_future_dataframe(periods=horizon, freq="MS")
        # # forecast = m.predict(future)

        # # Отбираем только наши горизонты
        # fc = forecast.set_index("ds")["yhat"].loc[test.index].values
        actual = test.values

        # Расчет метрик
        rmse  = np.sqrt(mean_squared_error(actual, fc))
        mae   = mean_absolute_error(actual, fc)
        mape  = (np.abs((actual - fc) / actual).mean()) * 100

        results_prophet.append({
            "Регион":     region,
            "Test start": test_start.strftime("%Y-%m"),
            "Test end":   test_end.strftime("%Y-%m"),
            "RMSE":       round(rmse, 2),
            "MAE":        round(mae, 2),
            "MAPE_%":     round(mape, 2),
            "Forecast":   [round(x, 2) for x in fc],
            "Actual":     [round(x, 2) for x in actual]
        })

# Собираем результаты в DataFrame
res_prophet = pd.DataFrame(results_prophet)
print("Результаты прогнозов Prophet на 3 месяца:")
display(res_prophet)

final_prophet = (
    res_prophet
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики Prophet по регионам (rolling-3):")
display(final_prophet)


15:07:38 - cmdstanpy - INFO - Chain [1] start processing
15:07:39 - cmdstanpy - INFO - Chain [1] done processing
15:07:39 - cmdstanpy - INFO - Chain [1] start processing
15:07:40 - cmdstanpy - INFO - Chain [1] done processing
15:07:40 - cmdstanpy - INFO - Chain [1] start processing
15:07:40 - cmdstanpy - INFO - Chain [1] done processing
15:07:40 - cmdstanpy - INFO - Chain [1] start processing
15:07:40 - cmdstanpy - INFO - Chain [1] done processing
15:07:40 - cmdstanpy - INFO - Chain [1] start processing
15:07:41 - cmdstanpy - INFO - Chain [1] done processing
15:07:41 - cmdstanpy - INFO - Chain [1] start processing
15:07:41 - cmdstanpy - INFO - Chain [1] done processing
15:07:41 - cmdstanpy - INFO - Chain [1] start processing
15:07:41 - cmdstanpy - INFO - Chain [1] done processing
15:07:42 - cmdstanpy - INFO - Chain [1] start processing
15:07:42 - cmdstanpy - INFO - Chain [1] done processing
15:07:42 - cmdstanpy - INFO - Chain [1] start processing
15:07:42 - cmdstanpy - INFO - Chain [1]

Результаты прогнозов Prophet на 3 месяца:


,Регион,Test start,Test end,RMSE,MAE,MAPE_%,Forecast,Actual
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,1416.91,1133.93,11.74,"[11017.33, 9581.39, 12312.82]","[8963.23, 9586.12, 10969.85]"
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,1081.75,913.22,8.45,"[9387.06, 11891.14, 9275.67]","[9586.12, 10969.85, 10894.97]"
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,1087.12,979.64,9.04,"[12026.85, 9380.67, 10606.59]","[10969.85, 10894.97, 10238.97]"
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,1372.88,1209.78,10.40,"[9324.72, 10537.5, 10931.91]","[10894.97, 10238.97, 12692.46]"
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,964.16,684.26,5.66,"[10650.87, 11074.23, 10356.51]","[10238.97, 12692.46, 10379.17]"
...,...,...,...,...,...,...,...,...
195,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,368.01,327.71,52.00,"[413.44, 229.12, 338.27]","[305.92, 742.4, 700.6]"
196,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,416.74,410.63,56.99,"[231.26, 339.86, 351.11]","[742.4, 700.6, 711.12]"
197,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,345.30,343.66,48.98,"[329.85, 347.11, 395.52]","[700.6, 711.12, 691.74]"
198,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,312.74,307.24,44.67,"[400.05, 457.87, 290.52]","[711.12, 691.74, 667.3]"


Средние метрики Prophet по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,1296.38,1087.96,10.53
1,АКТЮБИНСКАЯ ОБЛАСТЬ,39.21,33.55,208.73
2,АЛМАТИНСКАЯ ОБЛАСТЬ,1693.22,1545.61,13.17
3,АТЫРАУСКАЯ ОБЛАСТЬ,3.44,2.00,NaN
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,564.69,519.04,8.53
5,ГАЛМАТЫ,0.10,0.10,99.99
6,ГАСТАНА,0.02,0.01,NaN
7,ГШЫМКЕНТ,78.38,62.99,1075.22
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,451.89,391.54,23.15
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,335.36,270.03,27.94


In [10]:
# Переименуем колонки с MAPE, чтобы было понятно, к какому методу относятся
hw = final_hw.rename(columns={"MAPE_%": "MAPE_HW"})
sar = final_sarima.rename(columns={"MAPE_%": "MAPE_SARIMA"})
pr  = final_prophet.rename(columns={"MAPE_%": "MAPE_Prophet"})

# Мёрджим по региону
summary = (
    hw[["Регион", "MAPE_HW"]]
    .merge(sar[["Регион", "MAPE_SARIMA"]], on="Регион")
    .merge(pr[["Регион", "MAPE_Prophet"]], on="Регион")
)

# Определяем для каждой строки, какой столбец MAPE минимален
# idxmin вернёт название столбца с минимальным значением
summary["Best_method"] = summary[["MAPE_HW","MAPE_SARIMA","MAPE_Prophet"]] \
                           .idxmin(axis=1) \
                           .str.replace("MAPE_","")  # убираем префикс для красоты

# Если нужно, можно сразу отсортировать
# summary = summary.sort_values("Best_method")

# допустим, у вас уже есть summary
summary = summary.round({
    "MAPE_HW": 2,
    "MAPE_SARIMA": 2,
    "MAPE_Prophet": 2
})

# Готово!
print(summary.to_string(index=False))


                        Регион  MAPE_HW                 MAPE_SARIMA  MAPE_Prophet Best_method
           АКМОЛИНСКАЯ ОБЛАСТЬ     9.37                        9.27         10.53      SARIMA
           АКТЮБИНСКАЯ ОБЛАСТЬ   384.26                      356.69        208.73     Prophet
           АЛМАТИНСКАЯ ОБЛАСТЬ     6.56                        9.93         13.17          HW
            АТЫРАУСКАЯ ОБЛАСТЬ      NaN   3505499728190416879616.00           NaN      SARIMA
ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ     8.36                        8.74          8.53          HW
                       ГАЛМАТЫ    54.28                      168.59         99.99          HW
                       ГАСТАНА      NaN     32418544302525849600.00           NaN      SARIMA
                      ГШЫМКЕНТ   423.80                      780.23       1075.22          HW
            ЖАМБЫЛСКАЯ ОБЛАСТЬ    17.50                       19.32         23.15          HW
 ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ    16.21                     